# **Demo of lsst.afw.display -- displaying images using the LSST DM Astronomical Framework library**

**Owner:** Brant Robertson ([@brantr](https://github.com/LSSTScienceCollaborations/DMStackClub/issues/new?body=@brantr))  
**Level:** Introductory  
**Last Verified to Run:** 2018-08-24   
**Verified Stack Release:** v16.0  

## **Learning Objectives:**

In this tutorial we will 

* Show how to access the `lsst.afw.display` routines.

* Use the LSST data Butler to access processed data and inspect it visually.

This tutorial is designed to help users get a brief feel for the `lsst.afw.display` library that enables the visual inspection of data. The [`lsst.afw` library](https://github.com/lsst/afw) provides an "Astronomical Framework" (afw) while the `lsst.daf.*` libraries (see, e.g., [daf_base](https://github.com/lsst/daf_base)) provides a Data Access Framework (daf). Both libraries are used in this tutorial, with the `lsst.daf.persistence` library used to access a calibrated exposure (calexp) and the `lsst.afw.display` library used to show the exposure image on the screen.

This tutorial made use of the [`LowSurfaceBrightness.ipynb` StackClub notebook](https://nbviewer.jupyter.org/github/LSSTScienceCollaborations/StackClub/blob/rendered/SourceDetection/LowSurfaceBrightness.nbconvert.ipynb) by [Alex Drlica-Wagner](https://github.com/LSSTScienceCollaborations/DMStackClub/issues/new?body=@kadrlica).

## **Step 0) Import Common Python Libraries**

The [`matplotlib`](https://matplotlib.org/), [`numpy`](http://www.numpy.org/), and [`astropy`](http://www.astropy.org/) libraries are widely used Python libraries for plotting, scientific computing, and astronomical data analysis. We will use these packages in common ways below, including the `matplotlib.pyplot` plotting sublibrary. We also import the [`warnings` library](https://docs.python.org/2/library/warnings.html) to prevent some routine warning messages from printing to the screen.

In [ ]:
#allow for matplotlib to create inline plots in our notebook
%matplotlib inline                   
import numpy as np                   #imports numpy with the alias np
import matplotlib.pyplot as plt      #imports matplotlib.pyplot as plt
import warnings                      #imports the warnings library

Let's go ahead and import from `astropy` the image stretch limits from the familiar `zscale()` function.

In [ ]:
from astropy.visualization import ZScaleInterval     #This function allows use to use the `zscale()` rescaling limits function familiar from, e.g., DS9, to adjust the image stretch.
zscale = ZScaleInterval()                            #create an alias to the `ZScaleInterval()` function

And let the kernel know that we're happy not to have some useful warnings printed during this tutorial.

In [ ]:
warnings.simplefilter("ignore", category=FutureWarning)   #prevent some helpful but ancillary warning messages from printing during LSST DM Release v16 calls
warnings.simplefilter("ignore", category=UserWarning)     #prevent some helpful but ancillary warning messages from printing during LSST DM Release v16 calls

As a last preparatory task, we set the parameters of `matplotlib.pyplot` to give us a large default size for an image.

In [ ]:
plt.rcParams['figure.figsize'] = (8.0, 8.0)               #set a large default size for our images

## **Step 1) Loading the LSST DM Stack**

To manipulate data, the LSST DM Stack provides a `Butler` that enables generic access routines to DM-generated data. For more information, see [the Data Butler entry in the LSST Software User Guide](https://confluence.lsstcorp.org/display/LSWUG/Data+Butler). In order to access a calibrated exposure from data stored in the format required by the LSST Data Butler, we must load the `lsst.daf.persistence` library to produce a Butler instance from the data.

In [ ]:
import lsst.daf.persistence as dafPersist     #load lsst.daf.persistence to gain access to a Butler instance

Next, we need to load the `lsst.afw.display` library to gain access to the image visualization routines we'd like to use.

In [ ]:
import lsst.afw.display     as afwDisplay     #load lsst.afw.display to gain access to image visualization routines.

## **Step 2) Importing Data to Visualize**

To plot an image to the screen, we must first load some data. In this tutorial, we will use the `Twinkles` simulated images available in the StackClub data repository. These data sit in the data directory `/project/shared/data/Twinkles_subset/output_data_v2` and contain a set of data produced in generating a calibrated exposure by the DM Stack. These data are organized in a structure that enables a DM Stack `Butler` instance to be generated and provide access to a single filter image (in this case `r` band), a specific detector raft (2,2), a specific sensor in the raft (`1,1`) and a specific visit (in this case, 235 -- note only one band is available per visit in this example).

Once we define a string that contains the data directory, we start the `Butler` instance using the `lsst.daf.persistence` library alias `dafPersist` and its `Butler` class. The `Butler` object is initialized with a string containing the data directory we wish to access. Running the cell may take a few moments.

In [ ]:
datadir = "/project/shared/data/Twinkles_subset/output_data_v2"   #our data directory containing the Twinkles data organized as Butler expects
butler = dafPersist.Butler(datadir)                               #create an instance of the Butler, which we call `butler`, with access to our data

With the `Butler` instance now generated using our data directory, we can retrieve the desired calibrated exposure by telling the butler which filter, raft, sensor, and visit we wish to view. To do this, we definie dictionary with the required information.

In [ ]:
# Grab a calexp of interest
dataId = {'filter': 'r', 'raft': '2,2', 'sensor': '1,1', 'visit': 235}          #Define a dictionary with the filter, raft, sensor, and visit we wish to view
calexp = butler.get('calexp', **dataId)                                         #retrieve the data using the `butler` instance and its function `get()`

## **Step 3) Use AFWDisplay to Visualize the Image**

Now, with a `Butler` instance defined and a calibrated exposure retrieved, we can use [`lsst.afw.display`](https://github.com/lsst/afw/tree/master/python/lsst/afw/display) to visualize the data.  The next task is to let AFWDisplay know that we want it to enroll `matplotlib` as our default display backend. To do this, we use the `setDefaultBackend()` function. Remember that we made an alias to `lsst.afw.display` called `afwDisplay`, so we'll use that to call `setDefaultBackend()`.

In [ ]:
afwDisplay.setDefaultBackend('matplotlib')  # Use lsst.afw.display with the matplotlib backend

We are now set to display the image. To do this, we:

* First create a `matplotlib.pyplot` figure using `plt.figure()` -- this will be familiar to anyone with experience using `matplotlib`.
* Then create an alias to the `lsst.afw.display.Display` method that will allow us to display the data to the screen.  This alias will be called `afw_display`.
* Before showing the data on the screen, we have to decide how to apply an image stretch given the data. The algorithm we'll use is `asinh` familiar from SDSS images, with a range of values set by `zscale`. To do this, we use the `scale()` function provided by `lsst.afw.display`. See the `scale()` function definition in the [`interface.py` file of the lsst.afw.display library](https://github.com/lsst/afw/blob/master/python/lsst/afw/display/interface.py).
* Finally, we can display the image. Do do this, we provide the `mtv()` method the `image` member of our calibrated image retrieved by the `butler`. We can then use `plt.show()` to display our figure.

All these tasks are best done within the same notebook cell.

In [ ]:
plt.figure()                            #create a matplotlib.pyplot figure
afw_display = afwDisplay.Display()      #get an alias to the lsst.afw.display.Display() method
afw_display.scale('asinh', 'zscale')    #set the image stretch algorithm and range
afw_display.mtv(calexp.image)           #load the image into the display
plt.show()                              #show the corresponding pyplot figure

**Congrats!** We've plotted an image using `lsst.afw.display`!

## **Step 4) More Information about lsst.afw.display**

To get some more information about `lsst.afw.display`, we can print the method list to see what's available. The next cell will print `lsst.afw.display` methods to the screen.

In [ ]:
method_list = [func for func in dir(afw_display) if callable(getattr(afw_display, func))]
print(method_list)

If you'd like to learn more about any given function, please see the [`lsst.afw.display` source code](https://github.com/lsst/afw/tree/master/python/lsst/afw/display).

You can also read the API documentation about the above functions using the Jupyter notebook `help()` function:

In [ ]:
help(afw_display.scale)

In [ ]:
help(afw_display.mtv)

### **Further Documentation**

If you'd like some more information on `lsst.afw.display`, please have a look at the following websites:

* [Info on image indexing conventions.](https://github.com/lsst/afw/blob/master/doc/lsst.afw.image/indexing-conventions.rst)  
* [afw.display Doxygen website](http://doxygen.lsst.codes/stack/doxygen/x_masterDoxyDoc/namespacelsst_1_1afw_1_1display.html)  
* [afw.display GitHub website](https://github.com/RobertLuptonTheGood/afw/tree/master/python/lsst/afw/display)  
* [The `pipelines.lsst.io` Getting Started on Image Display website.](https://pipelines.lsst.io/getting-started/display.html)